## Setup

In [0]:
# need to write a function to download all the files. maybe next time
!mkdir utils

mkdir: cannot create directory ‘utils’: File exists


In [1]:
import os
import numpy as np
import keras
import keras.backend as K
import tensorflow as tf
import threading
import google.colab as colab
import cv2
import zipfile
import pickle
import queue

from utils.file_utils import *
from utils.image_utils import *
from utils.generator_utils import *
from utils.tqdm_utils import *
from utils.keras_utils import *


import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


### Mount Google Drive

In [0]:
def mount_google_drive():
	'''
	# Functionality
		Mount google drive. Since colab does not save files, we want to make it easier to directly access files in google drive.
	# Arguments
		Nothing
	# Returns
		drive_root: the working directory mounted
	'''
	mount_directory = "/content/gdrive"
	drive = colab.drive
	drive.mount(mount_directory, force_remount=True)
	drive_root = mount_directory + "/" + list(filter(lambda x: x[0] != '.', os.listdir(mount_directory)))[0]
	return drive_root

In [3]:
ROOT_DIR =  mount_google_drive()
CHECKPOINT_ROOT = ROOT_DIR+ "/captioning/checkpoints/"
DATASET_DIR = ROOT_DIR + "/Dataset/"


def get_checkpoint_path(epoch=None):
    if epoch is None:
        return os.path.abspath(CHECKPOINT_ROOT + "weights")
    else:
        return os.path.abspath(CHECKPOINT_ROOT + "weights_{}".format(epoch))
      
# example of checkpoint dir
print(get_checkpoint_path(4))

Mounted at /content/gdrive
/content/gdrive/My Drive/captioning/checkpoints/weights_4


## Create Embeddings for images

In [0]:
# This preprocessing function should be in the notebook not in the utils
# Since we may want to try different config every time. Should be custimizable. 
def preprocess(raw_bytes, input_shape):
    img = decode_image_from_raw_bytes(raw_bytes)
    img = image_center_crop(img)  # take center crop
    img = cv2.resize(img, input_shape)  # resize for our model
    img = img.astype("float32")  # prepare for normalization
    img = keras.applications.inception_v3.preprocess_input(img)  # preprocess for model
    return img

### Set up the encoder part

In [5]:
# Set up the encoder part
reset_tf_session()
K.set_learning_phase(False)
# We can also try resnet, wide-resnet, etc...
encoder = keras.applications.InceptionV3(include_top=False)
encoder = keras.models.Model(encoder.inputs, keras.layers.GlobalAveragePooling2D()(encoder.output))

encoder.summary()

W0718 02:36:03.336229 140085923268480 deprecation_wrapper.py:119] From /content/utils/keras_utils.py:68: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0718 02:36:03.338262 140085923268480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:95: The name tf.reset_default_graph is deprecated. Please use tf.compat.v1.reset_default_graph instead.

W0718 02:36:03.338977 140085923268480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:98: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0718 02:36:03.398699 140085923268480 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:102: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0718 02:36:03.400226 140085923268

87916544/87910968 [==============================] - 3s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 3 0           batch_normalization_1[0][0]      
____________________________________________

In [0]:
# Multi-threading code inspired by coursera hse-aml
def encoder_inference(zip_fn, model, input_shape=(224, 224), batch_size=32):
    # queue for cropped images
    q = queue.Queue(maxsize=batch_size * 10)

    # when read thread put all images in queue
    read_thread_completed = threading.Event()

    # time for read thread to die
    kill_read_thread = threading.Event()

    def reading_thread(zip_fn):
        zf = zipfile.ZipFile(zip_fn)
        for fn in zf.namelist():
            if kill_read_thread.is_set():
                break
            if fn.split(".")[-1] == "jpg":
                buf = zf.read(fn)  # read raw bytes from zip for fn
                img = preprocess(buf, input_shape)
                
                while True:
                    try:
                        q.put((os.path.split(fn)[-1], img), timeout=1)  # put in queue
                    except queue.Full:
                        if kill_read_thread.is_set():
                            break
                        continue
                    break
        read_thread_completed.set()  # read all images

    # start reading thread
    t = threading.Thread(target=reading_thread, args=(zip_fn,))
    t.daemon = True
    t.start()

    img_fns = []
    img_embeddings = []
    batch_imgs = []

    try:
        while True:
            try:
                fn, img = q.get(timeout=1)
            except queue.Empty:
                if read_thread_completed.is_set():
                    break
                continue
            img_fns.append(fn)
            batch_imgs.append(img)
            if len(batch_imgs) == batch_size:
              batch_imgs = np.stack(batch_imgs, axis=0)
              batch_embeddings = model.predict(batch_imgs)
              img_embeddings.append(batch_embeddings)
              batch_imgs = []
              
            q.task_done()
        # process last batch
        if len(batch_imgs):
            batch_imgs = np.stack(batch_imgs, axis=0)
            batch_embeddings = model.predict(batch_imgs)
            img_embeddings.append(batch_embeddings)
    finally:
        kill_read_thread.set()
        t.join()

    q.join()

    img_embeddings = np.vstack(img_embeddings)
    return img_embeddings, img_fns

In [0]:
def save_pickle(obj, fn):
	'''
	# Functionality
		Save the data into pickle format
	# Arguments
		obj: the data object
		fn: the pickle file name
	# Returns
		Nothing. Just save to the file.
	'''
	with open(fn, "wb") as f:
		pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)


def read_pickle(fn):

	'''
	# Functionality
		Save the data into pickle format
	# Arguments
		fn: the pickle file name
	# Returns
		obj: the desired data object
	'''
	with open(fn, "rb") as f:
		return pickle.load(f)

In [0]:
train_img_embeds, train_img_fns = encoder_inference(
    DATASET_DIR + "train2014_sample.zip", encoder, input_shape=(299, 299))

save_pickle(train_img_fns, DATASET_DIR + "train_img_fns.pickle")
save_pickle(train_img_embeds, DATASET_DIR + "train_img_embeds.pickle")

In [0]:
val_img_embeds, val_img_fns = encoder_inference(
    DATASET_DIR + "val2014_sample.zip", encoder, input_shape=(299, 299))

save_pickle(val_img_fns, DATASET_DIR + "val_img_fns.pickle")
save_pickle(val_img_embeds, DATASET_DIR + "val_img_embeds.pickle")